Due to BRENDA's license, you have to download the database as a text file after accepting usage conditions [here](https://www.brenda-enzymes.org/download.php).

In [1]:
from brendapyrser import BRENDA
dataFile = 'data/brenda_2023_1.txt'
brenda = BRENDA(dataFile)

In [2]:
reaction_ec_numbers = []
reaction_ec_annotation = []
reaction_names = []
reaction_systematic_name = []
reaction_str = []
reaction_species = []
reaction_reference = []
reaction_type = []
reaction_substrates = []
reaction_products = []
data = []

In [3]:
from utils import extract_substrates,extract_products, extract_just_ec_number, extract_ec_annotation
import json
for r in brenda.reactions:
    reaction_ec_numbers.append(extract_just_ec_number(r.ec_number))
    reaction_ec_annotation.append(extract_ec_annotation(r.ec_number))
    reaction_names.append(r.name)
    reaction_systematic_name.append(r.systematic_name)
    reaction_str.append(r.reaction_str)
    reaction_species.append(r.getSpeciesDict())
    reaction_reference.append(r.getReferencesDict())
    reaction_type.append(r.reaction_type)
    reaction_substrates.append(extract_substrates(r.substratesAndProducts))
    reaction_products.append(extract_products(r.substratesAndProducts))
    item = {
        'ec_number': extract_just_ec_number(r.ec_number),
        'ec_annotation': extract_ec_annotation(r.ec_number),
        'name': r.name,
        'systematic_name': r.systematic_name,
        'str': r.reaction_str,
        'species': json.dumps(r.getSpeciesDict()),
        'reference': json.dumps(r.getReferencesDict()),
        'type': r.reaction_type,
        'substrates': json.dumps(extract_substrates(r.substratesAndProducts)),
        'products': json.dumps(extract_products(r.substratesAndProducts))
    }
    for key in item.keys():
        if not item[key]:
            item[key] = None
    data.append(item)

In [6]:
# with mysql running
import pymysql
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '123456',
    'charset':'utf8mb4'
}
try:
    connection = pymysql.connect(**db_config)
    cursor = connection.cursor()
except pymysql.Error as e:
    print("Database connection error:", e)
try:
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS Reaction")
    cursor.execute(f"USE Reaction")
    cursor.execute("""
            CREATE TABLE IF NOT EXISTS reactions (
                ec_number VARCHAR(20) PRIMARY KEY, 
                ec_annotation TEXT,
                name VARCHAR(100),
                systematic_name VARCHAR(100),
                str TEXT,
                species JSON,
                reference JSON,
                type VARCHAR(100),
                substrates JSON,
                products JSON
            ) CHARACTER SET utf8;""")
    for item in data:
        try:
            insert_query = f"INSERT INTO reactions ({', '.join(item.keys())}) VALUES ({', '.join(['%s'] * len(item))})"
            cursor.execute(insert_query, tuple(item.values()))
        except pymysql.Error as e:
            print(e)
            
    cursor.close()
    connection.close()
except pymysql.Error as e:
    print("数据库连接错误:", e)

(1048, "Column 'ec_number' cannot be null")
(1366, "Incorrect string value: '\\xC2\\x96)-(1...' for column 'systematic_name' at row 1")
(1366, "Incorrect string value: '\\xC2\\x96)-bu...' for column 'systematic_name' at row 1")
(1366, "Incorrect string value: '\\xC2\\x96)-de...' for column 'systematic_name' at row 1")
(1366, "Incorrect string value: '\\xC2\\x97hemo...' for column 'systematic_name' at row 1")
(1366, "Incorrect string value: '\\xC2\\x96)-ve...' for column 'systematic_name' at row 1")
(1366, "Incorrect string value: '\\xC2\\x97hemo...' for column 'systematic_name' at row 1")
(1366, "Incorrect string value: '\\xC2\\x97hemo...' for column 'systematic_name' at row 1")
(1366, "Incorrect string value: '\\xC2\\x97hemo...' for column 'systematic_name' at row 1")
(1366, "Incorrect string value: '\\xC2\\x97hemo...' for column 'systematic_name' at row 1")
(1366, "Incorrect string value: '\\xC2\\x97hemo...' for column 'systematic_name' at row 1")
(1366, "Incorrect string value: '\\x

In [ ]:
brenda.reactions[2].substratesAndProducts

In [ ]:
m = 0
for i in reaction_type:
    try:
        m = max(m,len(i))
    except Exception as e:
        print(e)

In [4]:
data[11]

{'ec_number': '1.1.1.108',
 'ec_annotation': None,
 'name': 'Carnitine 3-dehydrogenase',
 'systematic_name': 'carnitine:NAD+ 3-oxidoreductase',
 'str': 'carnitine + NAD+ <=> 3-dehydrocarnitine + NADH + H+',
 'species': '{"1": {"name": "Bacteria", "proteinID": "", "refs": ["12"]}, "2": {"name": "Pseudomonas aeruginosa", "proteinID": "", "refs": ["4", "7", "8", "9", "10", "13"]}, "3": {"name": "Pseudomonas putida", "proteinID": "", "refs": ["1", "5", "7", "11"]}, "4": {"name": "Alcaligenes sp.", "proteinID": "", "refs": ["1"]}, "5": {"name": "Xanthomonas campestris", "proteinID": "", "refs": ["14"]}, "6": {"name": "Rhizobium sp.", "proteinID": "", "refs": ["16"]}, "7": {"name": "Agrobacterium sp.", "proteinID": "", "refs": ["1", "2"]}, "8": {"name": "Rhizobium sp. YS-240", "proteinID": "", "refs": ["3"]}, "9": {"name": "Xanthomonas translucens", "proteinID": "", "refs": ["6", "15", "16"]}, "10": {"name": "Pseudomonas aeruginosa", "proteinID": "Q9HTH8 UniProt", "refs": ["13"]}, "11": {"na